In [1]:
import pandas as pd

In [2]:
import openai

with open("../openai_api_key.txt", "r") as f:
    api_key = f.read()

GPT_model = "gpt-4"

In [3]:
RANDOM_SEED = 123

client = openai.Client(api_key=api_key)

In [8]:
# read descriptions from json file
from data_load import load_games, load_system_prompt, load_query_games

games = load_query_games()
system_prompt = load_system_prompt('naive_prompt.txt')

In [9]:
def get_model_response(message):
    completion = client.chat.completions.create(
    model=GPT_model,
    seed=RANDOM_SEED,
    messages=message,
    )
    return completion.choices[0].message.content

In [6]:
for game in games:
    path_name = game.code_file_path.split('/')[-1]
    game_tag = path_name[:-4]
    df = pd.read_csv(f'data/game_condition_query/{game_tag}.csv')
    df = df[df.columns[1:]]
    n_phrases_language = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
    N_PHRASES = n_phrases_language

    model_response = []
    for _, row in df.iterrows():
        messages = []
        messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": game.description})
        for i in range(1, N_PHRASES + 1):
            if not pd.isna(row[f"language_phrase_{i}"]):
                messages.append({"role": "user", "content": row[f"language_phrase_{i}"]})
        messages.append({"role": "user", "content": row["question_sentence"]})
        model_response.append(get_model_response(messages))

    output_df = df[['task_id', 'answer']].copy()
    output_df['model_answer'] = model_response
    output_df.to_csv(f'{GPT_model}_predictions/naive/{game_tag}.csv')